In [1]:
from utils import load_config
import os
os.chdir('/pfs/data5/home/ma/ma_ma/ma_kreffert/Interpretable-Deep-Fake-Detection/training')
print(os.getcwd())
from detectors import DETECTOR
from torchinfo import summary

path = "./config/detector/vit_bcos.yaml"
additional_args = {'test_batchSize': 4, 'pretrained':None}
config = load_config(path, additional_args=additional_args)

model_class = DETECTOR[config['model_name']]
model = model_class(config)

print(summary(model, depth=4))#input_size=(6, 224, 224)))

/pfs/data5/home/ma/ma_ma/ma_kreffert/Interpretable-Deep-Fake-Detection/training


2025-03-31 17:39:38.289652: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-31 17:39:38.329541: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-31 17:39:38.329574: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-31 17:39:38.329597: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-31 17:39:38.337841: I tensorflow/core/platform/cpu_feature_g

unknown module type <class 'torch.nn.modules.normalization.LayerNorm'>
unknown module type <class 'torch.nn.modules.normalization.LayerNorm'>
unknown module type <class 'torch.nn.modules.normalization.LayerNorm'>
unknown module type <class 'torch.nn.modules.normalization.LayerNorm'>
unknown module type <class 'einops.layers.torch.Rearrange'>
unknown module type <class 'training.networks.base.vit.PosEmbSinCos2d'>
unknown module type <class 'torch.nn.modules.normalization.LayerNorm'>
unknown module type <class 'torch.nn.modules.activation.Softmax'>
unknown module type <class 'training.networks.base.vit.Attention'>
unknown module type <class 'torch.nn.modules.normalization.LayerNorm'>
unknown module type <class 'training.networks.base.vit.Attention'>
unknown module type <class 'torch.nn.modules.normalization.LayerNorm'>
unknown module type <class 'torch.nn.modules.activation.Softmax'>
unknown module type <class 'training.networks.base.vit.Attention'>
unknown module type <class 'torch.nn.m

In [10]:
import sys
import argparse
from train import main

# Filter out unwanted arguments passed by Jupyter
sys.argv = [arg for arg in sys.argv if not arg.startswith('-f')]

# Manually define the arguments you want to pass
args = ['--detector_path', '/home/ma/ma_ma/ma_kreffert/Interpretable-Deep-Fake-Detection/training/config/detector/vit_bcos.yaml']

# Extend sys.argv with your custom arguments
sys.argv.extend(args)

# Use argparse to parse the arguments
def parse_args():
    parser = argparse.ArgumentParser(description='Process some paths.')
    parser.add_argument('--detector_path', type=str, help='path to detector YAML file')
    parser.add_argument("--train_dataset", nargs="+")
    parser.add_argument("--test_dataset", nargs="+")
    parser.add_argument('--no-save_ckpt', dest='save_ckpt', action='store_false', default=True)
    parser.add_argument('--no-save_feat', dest='save_feat', action='store_false', default=True)
    parser.add_argument("--ddp", action='store_true', default=False)
    parser.add_argument('--task_target', type=str, default="", help='specify the target of current training task')
    return parser.parse_known_args()

# Call parse_args() to parse the known and unknown arguments
args, unknown = parse_args()

# Now you can pass the args to the main function
main()  # The main function will be called with the modified arguments

usage: ipykernel_launcher.py [-h] [--detector_path DETECTOR_PATH]
                             [--train_dataset TRAIN_DATASET [TRAIN_DATASET ...]]
                             [--test_dataset TEST_DATASET [TEST_DATASET ...]]
                             [--no-save_ckpt] [--no-save_feat] [--ddp]
                             [--task_target TASK_TARGET]
ipykernel_launcher.py: error: unrecognized arguments: -f /pfs/data5/home/ma/ma_ma/ma_kreffert/.local/share/jupyter/runtime/kernel-67c54567-f04b-4f90-9c4f-128b81e65819.json


SystemExit: 2

In [15]:
model2 = vitc_s_patch1_14()
print(summary(model2, depth=3))#input_size=(6, 224, 224)))

Layer (type:depth-idx)                             Param #
SimpleViT                                          --
├─Sequential: 1-1                                  --
│    └─Sequential: 2-1                             --
│    │    └─BcosConv2d: 3-1                        2,592
│    │    └─LayerNorm: 3-2                         48
│    │    └─Identity: 3-3                          --
│    │    └─BcosConv2d: 3-4                        41,472
│    │    └─LayerNorm: 3-5                         96
│    │    └─Identity: 3-6                          --
│    │    └─BcosConv2d: 3-7                        165,888
│    │    └─LayerNorm: 3-8                         192
│    │    └─Identity: 3-9                          --
│    │    └─BcosConv2d: 3-10                       663,552
│    │    └─LayerNorm: 3-11                        384
│    │    └─Identity: 3-12                         --
│    └─Rearrange: 2-2                              --
│    └─BcosLinear: 2-3                             --
│   

/scratch/slurm_tmpdir/job_25429926/ipykernel_239986/315545080.py:487: UserWarning: You are trying to use the entrypoints from `bcos.models.vit` directly.
This is strongly discouraged as it might cause unintended silent errors.
Prefer to use the entrypoints from `bcos.models.pretrained` or `torch.hub`.
See lines 17-29 of this file () for why.
  warnings.warn(


In [13]:
"""
Contains:

- Simple ViT
- Simple ViT-C (i.e. ViT with a convolutional stem)
- B-cos

Code taken from lucidrain's vit-pytorch:
https://github.com/lucidrains/vit-pytorch/blob/b3e90a265284ba4df00e19fe7a1fd97ba3e3c113/vit_pytorch/simple_vit.py

Paper references
----------------
- Simple ViT: https://arxiv.org/abs/2205.01580
- Simple ViT-C: https://arxiv.org/abs/2106.14881
- B-cos: https://arxiv.org/abs/2205.10268

Note
----
This is compatible with both a non-B-cos SimpleViT and a B-cos SimpleViT,
provided that the correct arguments are passed.

Warning
-------
It is strongly recommended to use the entrypoints defined from `bcos.models.pretrained`
or the `torch.hub` interface to load models, instead of using this directly.
Especially for B-cos models, as they require a LogitBias module at the end of the model,
which the entrypoints below do not include.
Feel free to open up an issue at https://github.com/B-cos/B-cos-v2 if you have any questions.
"""
from collections import OrderedDict
from typing import Any, Callable, List, Tuple, Union

import torch
from einops import rearrange
from einops.layers.torch import Rearrange
from torch import Tensor, nn

from bcos.modules.common import DetachableModule
from bcos.modules import BcosLinear

__all__ = [
    "SimpleViT",
    # entrypoints
    "vitc_ti_patch1_14",
    "vitc_s_patch1_14",
    "vitc_b_patch1_14",
    "vitc_l_patch1_14",
    "simple_vit_ti_patch16_224",
    "simple_vit_s_patch16_224",
    "simple_vit_b_patch16_224",
    "simple_vit_l_patch16_224",
]
# helpers


def exists(x: Any) -> bool:
    return x is not None


def pair(t: Any) -> Tuple[Any, Any]:
    return t if isinstance(t, tuple) else (t, t)


# classes
class PosEmbSinCos2d(nn.Module):
    def __init__(self, temperature: Union[int, float] = 10_000):
        super().__init__()
        self.temperature = temperature

    def forward(self, patches: Tensor) -> Tensor:
        h, w, dim = patches.shape[-3:]
        device = patches.device
        dtype = patches.dtype

        y, x = torch.meshgrid(
            torch.arange(h, device=device),
            torch.arange(w, device=device),
            indexing="ij",
        )
        assert (dim % 4) == 0, "feature dimension must be multiple of 4 for sincos emb"
        omega = torch.arange(dim // 4, device=device) / (dim // 4 - 1)
        omega = 1.0 / (self.temperature**omega)

        y = y.flatten()[:, None] * omega[None, :]
        x = x.flatten()[:, None] * omega[None, :]
        pe = torch.cat((x.sin(), x.cos(), y.sin(), y.cos()), dim=1)
        return pe.type(dtype)


class FeedForward(nn.Module):
    def __init__(
        self,
        dim: int,
        hidden_dim: int,
        linear_layer: Callable[..., nn.Module] = None,
        norm_layer: Callable[..., nn.Module] = None,
        act_layer: Callable[..., nn.Module] = None,
    ):
        assert exists(linear_layer), "Provide a linear layer class!"
        assert exists(norm_layer), "Provide a norm layer (compatible with LN) class!"
        assert exists(act_layer), "Provide a activation layer class!"

        super().__init__()
        self.net = nn.Sequential(
            OrderedDict(
                norm=norm_layer(dim),
                linear1=linear_layer(dim, hidden_dim),
                act=act_layer(),
                linear2=linear_layer(hidden_dim, dim),
            )
        )

    def forward(self, x: Tensor) -> Tensor:
        return self.net(x)


class Attention(DetachableModule):
    def __init__(
        self,
        dim: int,
        heads: int = 8,
        dim_head: int = 64,
        linear_layer: nn.Module = None,
        norm_layer: nn.Module = None,
    ):
        assert exists(linear_layer), "Provide a linear layer class!"
        assert exists(norm_layer), "Provide a norm layer (compatible with LN) class!"

        super().__init__()
        self.att = None

        n_lins = 3
        inner_dim = dim_head * heads
        self.heads = heads
        self.scale = dim_head**-0.5
        self.norm = norm_layer(dim)
        self.pos_info = None
        self.attention_biases = None

        self.attend = nn.Softmax(dim=-1)
        self.to_qkv = nn.Linear(dim, inner_dim * n_lins, bias=False)
        self.to_out = linear_layer(inner_dim, dim, bias=False)

    def forward(self, x: Tensor) -> Tensor:
        x = self.norm(x)
        qkv = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: rearrange(t, "b n (h d) -> b h n d", h=self.heads), qkv)

        if self.detach:  # detach dynamic linear weights
            q = q.detach()
            k = k.detach()
            # these are used for dynamic linear w (`attn` below)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale
        attn = self.attend(dots)

        out = torch.matmul(attn, v)
        out = rearrange(out, "b h n d -> b n (h d)")
        return self.to_out(out)


class Encoder(nn.Module):
    def __init__(
        self,
        dim: int,
        heads: int,
        dim_head: int,
        mlp_dim: int,
        linear_layer: Callable[..., nn.Module] = None,
        norm_layer: Callable[..., nn.Module] = None,
        act_layer: Callable[..., nn.Module] = None,
    ):
        assert exists(linear_layer), "Provide a linear layer class!"
        assert exists(norm_layer), "Provide a norm layer (compatible with LN) class!"
        assert exists(act_layer), "Provide a activation layer class!"

        super().__init__()

        self.attn = Attention(
            dim,
            heads=heads,
            dim_head=dim_head,
            linear_layer=linear_layer,
            norm_layer=norm_layer,
        )

        self.ff = FeedForward(
            dim,
            mlp_dim,
            linear_layer=linear_layer,
            norm_layer=norm_layer,
            act_layer=act_layer,
        )

    def forward(self, x: Tensor) -> Tensor:
        x = self.attn(x) + x
        x = self.ff(x) + x
        return x


class Transformer(nn.Sequential):
    def __init__(
        self,
        dim: int,
        depth: int,
        heads: int,
        dim_head: int,
        mlp_dim: int,
        linear_layer: Callable[..., nn.Module] = None,
        norm_layer: Callable[..., nn.Module] = None,
        act_layer: Callable[..., nn.Module] = None,
    ):
        assert exists(linear_layer), "Provide a linear layer class!"
        assert exists(norm_layer), "Provide a norm layer (compatible with LN) class!"
        assert exists(act_layer), "Provide a activation layer class!"

        layers_odict = OrderedDict()
        for i in range(depth):
            layers_odict[f"encoder_{i}"] = Encoder(
                dim=dim,
                heads=heads,
                dim_head=dim_head,
                mlp_dim=mlp_dim,
                linear_layer=linear_layer,
                norm_layer=norm_layer,
                act_layer=act_layer,
            )
        super().__init__(layers_odict)


class SimpleViT(nn.Module):
    def __init__(
        self,
        *,
        image_size: Union[int, Tuple[int, int]],
        patch_size: Union[int, Tuple[int, int]],
        num_classes: int,
        dim: int,
        depth: int,
        heads: int,
        mlp_dim: int,
        channels: int = 6,
        linear_layer: Callable[..., nn.Module] = None,
        norm_layer: Callable[..., nn.Module] = None,
        act_layer: Callable[..., nn.Module] = None,
        norm2d_layer: Callable[..., nn.Module] = None,
        conv2d_layer: Callable[..., nn.Module] = None,
        conv_stem: List[int] = None,  # Output channels for each layer of conv stem
        **kwargs,
    ):
        _warn_if_not_called_from_bcos_models_pretrained_or_torch_hub()
        super().__init__()
        _ = kwargs  # Ignore additional experiment parameters...
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)

        linear_layer = BcosLinear #(b=b, max_out=max_out) # in_features: int, out_features: int, bias: bool = False, device=None, dtype=None,
        conv2d_layer = BcosConv2d #(b=b, max_out=1) # vit_config.get('conv2d_layer', None)

        norm_layer = norms.NoBias(nn.LayerNorm) #norms.NoBias(norms.DetachablePositionNorm2d)#norms.NoBias(norms.BatchNormUncentered2d) #vit_config.get('norm_layer', None)
        norm2d_layer = norms.NoBias(nn.LayerNorm) #norms.NoBias(norms.DetachablePositionNorm2d)#vit_config.get('norm2d_layer', None)
        act_layer = nn.Identity #vit_config.get
        
        assert exists(linear_layer), "Provide a linear layer class!"
        assert exists(norm_layer), "Provide a norm layer (compatible with LN) class!"
        assert exists(act_layer), "Provide a activation layer class!"
        if conv_stem:
            assert exists(
                conv2d_layer
            ), "Provide a conv2d layer class when using conv_stem!"
            assert exists(
                norm2d_layer
            ), "Provide a norm2d layer class when using conv_stem!"

        assert (
            image_height % patch_height == 0 and image_width % patch_width == 0
        ), "Image dimensions must be divisible by the patch size."

        self.num_patches = (image_height // patch_height) * (image_width // patch_width)
        self.patch_dim = (
            (channels if conv_stem is None else conv_stem[-1])
            * patch_height
            * patch_width
        )
        stem = (
            dict()
            if conv_stem is None
            else dict(
                conv_stem=make_conv_stem(
                    channels, conv_stem, conv2d_layer, norm2d_layer, act_layer
                )
            )
        )
        self.to_patch_embedding = nn.Sequential(
            OrderedDict(
                **stem,
                rearrage=Rearrange(
                    "b c (h p1) (w p2) -> b h w (p1 p2 c)",
                    p1=patch_height,
                    p2=patch_width,
                ),
                linear=linear_layer(self.patch_dim, dim),
            )
        )
        self.positional_embedding = PosEmbSinCos2d()

        dim_head = dim // heads
        self.transformer = Transformer(
            dim,
            depth,
            heads,
            dim_head,
            mlp_dim,
            linear_layer=linear_layer,
            norm_layer=norm_layer,
            act_layer=act_layer,
        )

        self.to_latent = nn.Identity()
        self.linear_head = nn.Sequential(
            OrderedDict(
                norm=norm_layer(dim),
                linear=linear_layer(dim, num_classes),
            )
        )

    def forward(self, img):
        x = self.to_patch_embedding(img)
        pe = self.positional_embedding(x)
        x = rearrange(x, "b ... d -> b (...) d") + pe

        x = self.transformer(x)
        x = x.mean(dim=1)

        x = self.to_latent(x)
        return self.linear_head(x)


def make_conv_stem(
    in_channels: int,
    out_channels: List[int],
    conv2d_layer: Callable[..., nn.Module] = None,
    norm2d_layer: Callable[..., nn.Module] = None,
    act_layer: Callable[..., nn.Module] = None,
):
    """
    Following the conv stem design in Early Convolutions Help Transformers See Better (Xiao et al.)
    """
    model = []
    for outc in out_channels:
        conv = conv2d_layer(
            in_channels,
            outc,
            kernel_size=3,
            stride=(2 if outc > in_channels else 1),
            padding=1,
        )
        in_channels = outc
        norm = norm2d_layer(in_channels)
        act = act_layer()
        model += [conv, norm, act]
    return nn.Sequential(*model)


def vitc_ti_patch1_14(**kwargs):
    kwargs.setdefault("num_classes", 1_000)
    return SimpleViT(
        image_size=14,
        patch_size=1,
        depth=12
        - 1,  # Early convs. help transformers see better: reduce depth to account for conv stem for fairness
        dim=384 // 2,
        heads=6 // 2,
        mlp_dim=1536 // 2,
        conv_stem=[24, 48, 96, 192],
        **kwargs,
    )


def vitc_s_patch1_14(**kwargs):
    kwargs.setdefault("num_classes", 2)
    return SimpleViT(
        image_size=14,
        patch_size=1,
        depth=12
        - 1,  # Early convs. help transformers see better: reduce depth to account for conv stem for fairness
        dim=384,
        heads=6,
        mlp_dim=1536,
        conv_stem=[48, 96, 192, 384],
        **kwargs,
    )


def vitc_b_patch1_14(**kwargs):
    kwargs.setdefault("num_classes", 1_000)
    return SimpleViT(
        image_size=14,
        patch_size=1,
        depth=12
        - 1,  # Early convs. help transformers see better: reduce depth to account for conv stem for fairness
        dim=384 * 2,
        heads=6 * 2,
        mlp_dim=1536 * 2,
        conv_stem=[64, 128, 128, 256, 256, 512],
        **kwargs,
    )


def vitc_l_patch1_14(**kwargs):
    kwargs.setdefault("num_classes", 1_000)
    return SimpleViT(
        image_size=14,
        patch_size=1,
        depth=14
        - 1,  # Early convs. help transformers see better: reduce depth to account for conv stem for fairness
        dim=1024,
        heads=16,
        mlp_dim=1024 * 4,
        conv_stem=[64, 128, 128, 256, 256, 512],
        **kwargs,
    )


def simple_vit_s_patch16_224(**kwargs):
    kwargs.setdefault("num_classes", 1_000)
    return SimpleViT(
        image_size=224,
        patch_size=16,
        dim=384,
        depth=12,
        heads=6,
        mlp_dim=1536,
        **kwargs,
    )


def simple_vit_ti_patch16_224(**kwargs):
    kwargs.setdefault("num_classes", 1_000)
    return SimpleViT(
        image_size=224,
        patch_size=16,
        dim=384 // 2,
        heads=6 // 2,
        mlp_dim=1536 // 2,
        depth=12,
        **kwargs,
    )


def simple_vit_b_patch16_224(**kwargs):
    kwargs.setdefault("num_classes", 1_000)
    return SimpleViT(
        image_size=224,
        patch_size=16,
        depth=12,
        dim=384 * 2,
        heads=6 * 2,
        mlp_dim=1536 * 2,
        **kwargs,
    )


def simple_vit_l_patch16_224(**kwargs):
    kwargs.setdefault("num_classes", 1_000)
    return SimpleViT(
        image_size=224,
        patch_size=16,
        depth=14,
        dim=1024,
        heads=16,
        mlp_dim=1024 * 4,
        **kwargs,
    )


def _warn_if_not_called_from_bcos_models_pretrained_or_torch_hub():
    """
    Warns the user if this module is not called from bcos.models.pretrained or torch.hub
    """
    import inspect
    import warnings

    # if this file is not called from bcos.models.pretrained or torch.hub, warn the user
    # note: hubconf uses bcos.models.pretrained under the hood
    if not any("pretrained" in call.filename for call in inspect.stack()):
        warnings.warn(
            "You are trying to use the entrypoints from `bcos.models.vit` directly.\n"
            "This is strongly discouraged as it might cause unintended silent errors.\n"
            "Prefer to use the entrypoints from `bcos.models.pretrained` or `torch.hub`.\n"
            f"See lines 17-29 of this file () for why."
        )

# VGG

In [1]:
from utils import load_config
import os
os.chdir('/pfs/data5/home/ma/ma_ma/ma_kreffert/Interpretable-Deep-Fake-Detection/training')
print(os.getcwd())
from detectors import DETECTOR
from torchinfo import summary

path = "./config/detector/vgg_bcos.yaml"
additional_args = {'test_batchSize': 4, 'pretrained':False}
config = load_config(path, additional_args=additional_args)

model_class = DETECTOR[config['model_name']]
model = model_class(config)

print(summary(model, depth=4))#input_size=(6, 224, 224)))

/pfs/data5/home/ma/ma_ma/ma_kreffert/Interpretable-Deep-Fake-Detection/training


2025-03-31 16:59:02.997947: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-31 16:59:03.036265: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-31 16:59:03.036296: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-31 16:59:03.036320: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-31 16:59:03.044238: I tensorflow/core/platform/cpu_feature_g

Registrierte Backbones: dict_keys(['inception_bcos', 'xception', 'resnet34', 'resnet34_bcos_v2', 'resnet34_bcos_v2_minimal', 'vit_bcos', 'convnext_bcos', 'vgg11_v2_bcos'])
Layer (type:depth-idx)                   Param #
VGGBcosDetector                          --
├─BcosVGG: 1-1                           --
│    └─Sequential: 2-1                   --
│    │    └─BcosConv2d: 3-1              --
│    │    │    └─NormedConv2d: 4-1       3,456
│    │    └─Identity: 3-2                --
│    │    └─AvgPool2d: 3-3               --
│    │    └─BcosConv2d: 3-4              --
│    │    │    └─NormedConv2d: 4-2       73,728
│    │    └─Identity: 3-5                --
│    │    └─AvgPool2d: 3-6               --
│    │    └─BcosConv2d: 3-7              --
│    │    │    └─NormedConv2d: 4-3       294,912
│    │    └─Identity: 3-8                --
│    │    └─BcosConv2d: 3-9              --
│    │    │    └─NormedConv2d: 4-4       589,824
│    │    └─Identity: 3-10               --
│    │    └─Av

In [3]:
model2 = vgg11(pretrained=False, progress=True, num_classes=2)
print(summary(model2, depth=4))#input_size=(6, 224, 224)))

Layer (type:depth-idx)                   Param #
BcosVGG                                  --
├─Sequential: 1-1                        --
│    └─BcosConv2d: 2-1                   --
│    │    └─NormedConv2d: 3-1            3,456
│    └─Identity: 2-2                     --
│    └─AvgPool2d: 2-3                    --
│    └─BcosConv2d: 2-4                   --
│    │    └─NormedConv2d: 3-2            73,728
│    └─Identity: 2-5                     --
│    └─AvgPool2d: 2-6                    --
│    └─BcosConv2d: 2-7                   --
│    │    └─NormedConv2d: 3-3            294,912
│    └─Identity: 2-8                     --
│    └─BcosConv2d: 2-9                   --
│    │    └─NormedConv2d: 3-4            589,824
│    └─Identity: 2-10                    --
│    └─AvgPool2d: 2-11                   --
│    └─BcosConv2d: 2-12                  --
│    │    └─NormedConv2d: 3-5            1,179,648
│    └─Identity: 2-13                    --
│    └─BcosConv2d: 2-14                  --
│  

In [2]:
"""
B-cos VGG models

Modified from https://github.com/pytorch/vision/blob/0504df5ddf9431909130e7788faf05446bb8a2/torchvision/models/vgg.py
"""
import math
from typing import Any, Callable, Dict, List, Optional, Union, cast

import torch
import torch.nn as nn
import torch.nn.functional as F

from bcos.common import BcosUtilMixin
from bcos.modules import BcosConv2d, LogitLayer, norms

__all__ = [
    "BcosVGG",
    "vgg11",
    "vgg11_bnu",
    "vgg13",
    "vgg13_bnu",
    "vgg16",
    "vgg16_bnu",
    "vgg19_bnu",
    "vgg19",
]


DEFAULT_CONV_LAYER = BcosConv2d
DEFAULT_NORM_LAYER = norms.NoBias(norms.BatchNormUncentered2d)


class BcosVGG(BcosUtilMixin, nn.Module):
    def __init__(
        self,
        features: nn.Module,
        num_classes: int = 1000,
        init_weights: bool = True,
        conv_layer: Callable[..., nn.Module] = DEFAULT_CONV_LAYER,
        logit_bias: Optional[float] = None,
        logit_temperature: Optional[float] = None,
    ) -> None:
        super(BcosVGG, self).__init__()
        self.features = features
        # self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            conv_layer(512, 4096, kernel_size=7, padding=3, scale_fact=1000),
            # nn.ReLU(True),
            # nn.Dropout(),
            conv_layer(4096, 4096, scale_fact=1000),
            # nn.ReLU(True),
            # nn.Dropout(),
            conv_layer(4096, num_classes, scale_fact=1000),
        )
        self.num_classes = num_classes
        self.logit_layer = LogitLayer(
            logit_temperature=logit_temperature,
            logit_bias=logit_bias or -math.log(num_classes - 1),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        # x = self.avgpool(x)
        # x = torch.flatten(x, 1)[..., None, None]
        x = self.classifier(x)
        # because it's supposed to come after
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = torch.flatten(x, 1)
        x = self.logit_layer(x)
        return x

    def get_classifier(self) -> nn.Module:
        """Returns the classifier part of the model. Note this comes before global pooling."""
        return self.classifier

    def get_feature_extractor(self) -> nn.Module:
        """Returns the feature extractor part of the model. Without global pooling."""
        return self.features

    def _initialize_weights(self) -> None:
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)


def make_layers(
    cfg: List[Union[str, int]],
    norm_layer: Callable[..., nn.Module] = DEFAULT_NORM_LAYER,
    conv_layer: Callable[..., nn.Module] = DEFAULT_CONV_LAYER,
    in_channels: int = 6,
    no_pool=False,
) -> nn.Sequential:
    layers: List[nn.Module] = []
    new_config = []
    for idx, entry in enumerate(cfg):
        new_config.append([entry, 1])
        if entry == "M" and no_pool:
            new_config[idx - 1][1] = 2

    for v, stride in new_config:
        if v == "M":
            if no_pool:
                continue
            layers += [nn.AvgPool2d(kernel_size=2, stride=2)]
        else:
            v = cast(int, v)
            conv2d = conv_layer(
                in_channels, v, kernel_size=3, padding=1, stride=stride, scale_fact=1000
            )
            if not isinstance(norm_layer, nn.Identity):
                layers += [conv2d, norm_layer(v)]  # , nn.ReLU(inplace=True)]
            else:
                layers += [conv2d]
            in_channels = v
    return nn.Sequential(*layers)


cfgs: Dict[str, List[Union[str, int]]] = {
    "A": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "B": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "D": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        "M",
    ],
    "E": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        512,
        "M",
    ],
}


def _vgg(
    arch: str,
    cfg: str,
    pretrained: bool,
    progress: bool,
    norm_layer: Callable[..., nn.Module] = DEFAULT_NORM_LAYER,
    conv_layer: Callable[..., nn.Module] = DEFAULT_CONV_LAYER,
    in_chans: int = 6,
    no_pool=False,
    **kwargs: Any
) -> BcosVGG:
    if pretrained:
        kwargs["init_weights"] = False
    model = BcosVGG(
        make_layers(
            cfgs[cfg],
            norm_layer=norm_layer,
            conv_layer=conv_layer,
            in_channels=in_chans,
            no_pool=no_pool,
        ),
        conv_layer=conv_layer,
        **kwargs,
    )
    if pretrained:
        raise ValueError(
            "If you want to load pretrained weights, then please use the entrypoints in "
            "bcos.pretrained or bcos.model.pretrained instead."
        )
    return model


def vgg11(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> BcosVGG:
    return _vgg("vgg11", "A", pretrained, progress, norm_layer=nn.Identity, **kwargs)


def vgg11_bnu(
    pretrained: bool = False, progress: bool = True, **kwargs: Any
) -> BcosVGG:
    return _vgg("vgg11_bnu", "A", pretrained, progress, **kwargs)


def vgg13(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> BcosVGG:
    return _vgg("vgg13", "B", pretrained, progress, norm_layer=nn.Identity, **kwargs)


def vgg13_bnu(
    pretrained: bool = False, progress: bool = True, **kwargs: Any
) -> BcosVGG:
    return _vgg("vgg13_bnu", "B", pretrained, progress, **kwargs)


def vgg16(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> BcosVGG:
    return _vgg("vgg16", "D", pretrained, progress, norm_layer=nn.Identity, **kwargs)


def vgg16_bnu(
    pretrained: bool = False, progress: bool = True, **kwargs: Any
) -> BcosVGG:
    return _vgg("vgg16_bnu", "D", pretrained, progress, **kwargs)


def vgg19(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> BcosVGG:
    return _vgg("vgg19", "E", pretrained, progress, norm_layer=nn.Identity, **kwargs)


def vgg19_bnu(
    pretrained: bool = False, progress: bool = True, **kwargs: Any
) -> BcosVGG:
    return _vgg("vgg19_bnu", "E", pretrained, progress, **kwargs)